In [1]:
import copy
import math

from PuzzlePiece import *
from SWPuzzleAligner import *
from NWPuzzleAligner import *
from EuclideanSimilarity import *

In [2]:
# gather all the pieces
target_pieces = {}

for p in ['C','N']:#,'NE','E','SE','S','SW','W','NW']:
    target_pieces[p] = PuzzlePiece('3x3_pieces/' + p + '_border.csv', 
                                   border_sampling_rate = .1)

In [3]:
# get the first piece
query = target_pieces.pop('C') # this piece serves as the core

In [4]:
###############################
# REPEAT HERE
##############################
print(target_pieces.keys())

dict_keys(['N'])


In [104]:
# find the closest match in the other pieces
EucSim = EuclideanSimilarity()
SWAligner = SWPuzzleAligner(EucSim)
NWAligner = NWPuzzleAligner(EucSim)

mx_score = 0
mx_piece = ''

all_align = {'rough':{}, 'fine':{}}
rough_piece_align = {}
fine_piece_align = {}

rough_alignments_count = 2

for i,p in enumerate(target_pieces):
    
    target = target_pieces[p]
    
    # extend the query to account for circular sequence
    tail_length = min(int(len(target_pieces[p].border_sample) * .25),
                      int(len(query.border_sample) * .25))

    query.extend_border_sample(tail_length)
    target.extend_border_sample(tail_length, reverse = True)
    
    # do a rough alignment with the extended border sample
    rough_piece_align = SWAligner.Align(Q = query.border_sample_ext, 
                                        T = target.border_sample_ext,
                                        window = 5, cutoff_percentile = 0.05,
                                        return_top = rough_alignments_count) 
    
    if max(rough_piece_align['mx']) > mx_score:
        mx_score = max(rough_piece_align['mx'])
        mx_piece = p
    
    all_align['rough'][p] = rough_piece_align
    all_align['rough'][p]['fine_dist'] = {}
    
    # for each rough alignent, do a fine alignment
    for i in range(rough_alignments_count):
        
        # plot the current rough alignment
        mx = all_align['rough'][p]['mx'][i]
        mx_Q = all_align['rough'][p]['mx_Q'][i]
        mx_T = all_align['rough'][p]['mx_T'][i]
        length = all_align['rough'][p]['length'][i]
        
        # find the aligned points window
        Q_pt = query.border_sample[query.ext_to_old_index[mx_Q]]
        T_pt = target.border_sample[target.ext_to_old_index[mx_T]] 

        # find x,y shift to align pieces based on that one point
        T_xshift = T_pt['x'] - Q_pt['x']
        T_yshift = T_pt['y'] - Q_pt['y'] 

#         plt.figure(figsize = [10,10])
#         plt.scatter( [query.border_sample[k]['x'] for k,v in query.border_sample.items() ],
#                      [query.border_sample[k]['y'] for k,v in query.border_sample.items() ])
#         plt.scatter( [target.border_sample[k]['x'] - T_xshift for k,v in target.border_sample.items() ],
#                      [target.border_sample[k]['y'] - T_yshift for k,v in target.border_sample.items() ])

#         # show best matched points - second will overplot first due to alignment
#         plt.scatter( Q_pt['x'], Q_pt['y'], s=400)
#         plt.scatter( T_pt['x'] - T_xshift, T_pt['y'] - T_yshift, s=196 ) 

#         # show similarity window
#         # the black and grey points represent the points in the positive scoring diagonal
#         # of the suffix table starting at the maximum scoring point
#         # these points are the best locally aligned points
#         Q_window = [query.border_sample[query.ext_to_old_index[q]] for q in range(mx_Q - length, mx_Q)]
#         plt.scatter( [p['x'] for p in Q_window],
#                      [p['y'] for p in Q_window], c = 'grey' )

#         T_window = [target.border_sample[target.ext_to_old_index[t]] for t in range(mx_T - length, mx_T)]
#         plt.scatter( [p['x'] - T_xshift for p in T_window],
#                      [p['y'] - T_yshift for p in T_window], c = 'black' )
#         plt.show()
        
        ####################
        # find start and stop matched points for the fine alignment
        
        q_stop = query.ext_to_old_index[rough_piece_align['mx_Q'][i]] + 2
        q_start = q_stop - rough_piece_align['length'][i] - 3
        
        if q_start < 0:
            q_start += len(query.border_sample)
        elif q_start >= len(query.border_sample):
            q_start -= len(query.border_sample)
            
        q_orig_start = query.border_sample[q_start]['orig_idx']
        q_orig_stop = query.border_sample[q_stop]['orig_idx']
        
        if q_orig_start < q_orig_stop:
            q_fine_seq = {i:query.ordered_border[b] for i,b in enumerate(range(q_orig_start, q_orig_stop,))}
        else:
            idx = [i for i in range(q_orig_start,len(query.ordered_border))]
            idx = idx + [i for i in range(0, q_orig_stop+1)]
            q_fine_seq = {i:query.ordered_border[b] for i,b in enumerate(idx)}
            
        t_start = target.ext_to_old_index[rough_piece_align['mx_T'][i]] - 2
        t_stop = t_start + rough_piece_align['length'][i] + 3
        
        if t_stop >= len(target.border_sample):
            t_stop -= len(target.border_sample)
            
        t_orig_start = target.border_sample[t_start]['orig_idx']
        t_orig_stop = target.border_sample[t_stop]['orig_idx']
        
        if t_orig_start < t_orig_stop:
            t_fine_seq = {i:target.ordered_border[b] for i,b in enumerate(range(t_orig_stop, t_orig_start, -1))}
        else:
            idx = [i for i in range(t_orig_stop,-1,-1)]
            idx = idx + [i for i in range(len(target.ordered_border)-1, t_orig_start-1, -1)]
            t_fine_seq = {i:target.ordered_border[b] for i,b in enumerate(idx)}
        
        min_dist = None
        slip = query.border_sample[1]['orig_idx'] - query.border_sample[0]['orig_idx']
        all_align['rough'][p]['fine_dist'][p+str(i)] = []
        for s in range(-slip, slip+1):
            if s < 0:
                q_slip = { k:q_fine_seq[k] for k in list(q_fine_seq.keys())[slip+s:len(q_fine_seq)+s] }
            elif s == 0:
                q_slip = { k:q_fine_seq[k] for k in list(q_fine_seq.keys())[slip:len(q_fine_seq)-slip] }
            else:
                q_slip = { k:q_fine_seq[k] for k in list(q_fine_seq.keys())[slip+s:len(q_fine_seq)-slip] }
            
            fine_dist = EucSim.ComputeSubseqDist(q_slip, t_fine_seq)
            
            print('slip', s)
            print('q_orig', q_slip[min(list(q_slip.keys()))], 
                            q_slip[max(list(q_slip.keys()))])
            print('q_size', q_slip[max(list(q_slip.keys()))]['order'] -
                            q_slip[min(list(q_slip.keys()))]['order'])
            print('t_orig', t_slip[min(list(t_slip.keys()))], 
                            t_slip[max(list(t_slip.keys()))])
            print('t_size', t_slip[max(list(t_slip.keys()))]['order'] -
                            t_slip[min(list(t_slip.keys()))]['order'])
            print('dist', fine_dist)
            
            all_align['rough'][p]['fine_dist'][p+str(i)].append(fine_dist)
        
        ########################
            
        # find the aligned points window
#         Q_pt_fine = q_fine_seq[fine_mx_Q]
#         T_pt_fine = t_fine_seq[fine_mx_T]

#          # find x,y shift to align pieces based on that one point
#         T_xshift = T_pt_fine['x'] - Q_pt_fine['x']
#         T_yshift = T_pt_fine['y'] - Q_pt_fine['y'] 

#         plt.figure(figsize = [10,10])
#         plt.scatter( [query.ordered_border[k]['x'] for k,v in query.ordered_border.items() ],
#                      [query.ordered_border[k]['y'] for k,v in query.ordered_border.items() ])
#         plt.scatter( [target.ordered_border[k]['x'] - T_xshift for k,v in target.ordered_border.items() ],
#                      [target.ordered_border[k]['y'] - T_yshift for k,v in target.ordered_border.items() ])

#         # show best matched points - second will overplot first due to alignment
#         plt.scatter( Q_pt_fine['x'], Q_pt_fine['y'], s=400)
#         plt.scatter( T_pt_fine['x'] - T_xshift, T_pt_fine['y'] - T_yshift, s=196 ) 

        # show similarity window
        # the black and grey points represent the points in the positive scoring diagonal
        # of the suffix table starting at the maximum scoring point
        # these points are the best locally aligned points
#         Q_window = [query.ordered_border[q] for q in range(mx_Q - length, mx_Q)]
#         plt.scatter( [p['x'] for p in Q_window],
#                      [p['y'] for p in Q_window], c = 'grey' )

#         T_window = [target.ordered_border[t] for t in range(mx_T - length, mx_T)]
#         plt.scatter( [p['x'] - T_xshift for p in T_window],
#                      [p['y'] - T_yshift for p in T_window], c = 'black' )
#         plt.show()
#             print(mx, mx_Q, mx_T, length)
            #########################################
            
            
# i = all_align['rough'][mx_piece]['mx'].index(mx_score)
# print('Best: ', mx_piece, mx_score, all_align['rough'][mx_piece]['mx_Q'][i], 
#        all_align['rough'][mx_piece]['mx_T'][i], all_align['rough'][mx_piece]['length'][i])
# #target = target_pieces.pop(mx_piece)['rough']
# target = target_pieces[mx_piece]


slip -10
q_orig {'x': 241, 'y': 62, 'order': 650} {'x': 68, 'y': 66, 'order': 919}
q_size 269
t_orig {'x': 220, 'y': 199, 'order': 400} {'x': 57, 'y': 188, 'order': 141}
t_size -259
dist 32.50332016286538
slip -9
q_orig {'x': 241, 'y': 63, 'order': 651} {'x': 67, 'y': 66, 'order': 920}
q_size 269
t_orig {'x': 220, 'y': 199, 'order': 400} {'x': 57, 'y': 188, 'order': 141}
t_size -259
dist 30.690976266847173
slip -8
q_orig {'x': 240, 'y': 64, 'order': 652} {'x': 66, 'y': 66, 'order': 921}
q_size 269
t_orig {'x': 220, 'y': 199, 'order': 400} {'x': 57, 'y': 188, 'order': 141}
t_size -259
dist 29.295414136256277
slip -7
q_orig {'x': 239, 'y': 64, 'order': 653} {'x': 65, 'y': 65, 'order': 922}
q_size 269
t_orig {'x': 220, 'y': 199, 'order': 400} {'x': 57, 'y': 188, 'order': 141}
t_size -259
dist 25.673358985042942
slip -6
q_orig {'x': 238, 'y': 65, 'order': 654} {'x': 64, 'y': 65, 'order': 923}
q_size 269
t_orig {'x': 220, 'y': 199, 'order': 400} {'x': 57, 'y': 188, 'order': 141}
t_size -259

In [100]:
print(q_start, q_stop, q_orig_start, q_orig_stop)
print(t_start, t_stop, t_orig_start, t_orig_stop)

print(q_fine_seq)
print()
print(t_fine_seq)

65 93 650 930
14 42 140 420
{0: {'x': 241, 'y': 62, 'order': 650}, 1: {'x': 241, 'y': 63, 'order': 651}, 2: {'x': 240, 'y': 64, 'order': 652}, 3: {'x': 239, 'y': 64, 'order': 653}, 4: {'x': 238, 'y': 65, 'order': 654}, 5: {'x': 237, 'y': 65, 'order': 655}, 6: {'x': 236, 'y': 66, 'order': 656}, 7: {'x': 235, 'y': 66, 'order': 657}, 8: {'x': 234, 'y': 66, 'order': 658}, 9: {'x': 233, 'y': 67, 'order': 659}, 10: {'x': 232, 'y': 67, 'order': 660}, 11: {'x': 231, 'y': 67, 'order': 661}, 12: {'x': 230, 'y': 68, 'order': 662}, 13: {'x': 229, 'y': 68, 'order': 663}, 14: {'x': 228, 'y': 68, 'order': 664}, 15: {'x': 227, 'y': 69, 'order': 665}, 16: {'x': 226, 'y': 69, 'order': 666}, 17: {'x': 225, 'y': 69, 'order': 667}, 18: {'x': 224, 'y': 69, 'order': 668}, 19: {'x': 223, 'y': 70, 'order': 669}, 20: {'x': 222, 'y': 70, 'order': 670}, 21: {'x': 221, 'y': 70, 'order': 671}, 22: {'x': 220, 'y': 70, 'order': 672}, 23: {'x': 219, 'y': 70, 'order': 673}, 24: {'x': 218, 'y': 70, 'order': 674}, 25: {'

In [ ]:
for i in range(len(all_align['rough'][mx_piece]['mx'])):
    mx = all_align['rough'][mx_piece]['mx'][i]
    mx_Q = all_align['rough'][mx_piece]['mx_Q'][i]
    mx_T = all_align['rough'][mx_piece]['mx_T'][i]
    length = all_align['rough'][mx_piece]['length'][i]

    # find the aligned points window
    Q_pt = query.border_sample[query.ext_to_old_index[mx_Q]]
    T_pt = target.border_sample[target.ext_to_old_index[mx_T]] 
    
    # find x,y shift to align pieces based on that one point
    T_xshift = T_pt['x'] - Q_pt['x']
    T_yshift = T_pt['y'] - Q_pt['y'] 

    plt.figure(figsize = [10,10])
    plt.scatter( [query.border_sample[k]['x'] for k,v in query.border_sample.items() ],
                 [query.border_sample[k]['y'] for k,v in query.border_sample.items() ])
    plt.scatter( [target.border_sample[k]['x'] - T_xshift for k,v in target.border_sample.items() ],
                 [target.border_sample[k]['y'] - T_yshift for k,v in target.border_sample.items() ])

    # show best matched points - second will overplot first due to alignment
    plt.scatter( Q_pt['x'], Q_pt['y'], s=400)
    plt.scatter( T_pt['x'] - T_xshift, T_pt['y'] - T_yshift, s=196 ) 

    # show similarity window
    # the black and grey points represent the points in the positive scoring diagonal
    # of the suffix table starting at the maximum scoring point
    # these points are the best locally aligned points
    Q_window = [query.border_sample[query.ext_to_old_index[q]] for q in range(mx_Q - length, mx_Q)]
    plt.scatter( [p['x'] for p in Q_window],
                 [p['y'] for p in Q_window], c = 'grey' )

    T_window = [target.border_sample[target.ext_to_old_index[t]] for t in range(mx_T - length, mx_T)]
    plt.scatter( [p['x'] - T_xshift for p in T_window],
                 [p['y'] - T_yshift for p in T_window], c = 'black' )
    plt.show()
    
    for j in range(len(all_align['fine'][mx_piece + str(i)])):
        mx = all_align['fine'][mx_piece + str(i)]['mx'][i]
        mx_Q = all_align['fine'][mx_piece + str(i)]['mx_Q'][i]
        mx_T = all_align['fine'][mx_piece + str(i)]['mx_T'][i]
        length = all_align['fine'][mx_piece + str(i)]['length'][i]
        
        # find the aligned points window
        Q_pt_fine = query.ordered_border[mx_Q]
        T_pt_fine = target.ordered_border[mx_T] 


        # find x,y shift to align pieces based on that one point
        T_xshift = T_pt_fine['x'] - Q_pt_fine['x']
        T_yshift = T_pt_fine['y'] - Q_pt_fine['y'] 

        plt.figure(figsize = [10,10])
        plt.scatter( [query.ordered_border[k]['x'] for k,v in query.ordered_border.items() ],
                     [query.ordered_border[k]['y'] for k,v in query.ordered_border.items() ])
        plt.scatter( [target.ordered_border[k]['x'] - T_xshift for k,v in target.ordered_border.items() ],
                     [target.ordered_border[k]['y'] - T_yshift for k,v in target.ordered_border.items() ])

        # show best matched points - second will overplot first due to alignment
        plt.scatter( Q_pt_fine['x'], Q_pt_fine['y'], s=400)
        plt.scatter( T_pt_fine['x'] - T_xshift, T_pt_fine['y'] - T_yshift, s=196 ) 

        # show similarity window
        # the black and grey points represent the points in the positive scoring diagonal
        # of the suffix table starting at the maximum scoring point
        # these points are the best locally aligned points
        Q_window = [query.ordered_border[q] for q in range(mx_Q - length, mx_Q)]
        plt.scatter( [p['x'] for p in Q_window],
                     [p['y'] for p in Q_window], c = 'grey' )

        T_window = [target.ordered_border[t] for t in range(mx_T - length, mx_T)]
        plt.scatter( [p['x'] - T_xshift for p in T_window],
                     [p['y'] - T_yshift for p in T_window], c = 'black' )
        plt.show()
        print(mx, mx_Q, mx_T, length)
        


In [ ]:
#########################
# REPEAT ABOVE
######################### 

# burn this one for now
target = target_pieces.pop(mx_piece)

In [ ]:
print(len(target.border_sample))

In [ ]:
# combine the pieces into one piece
Q_window_pts = [q for q in range(mx_Q-length, mx_Q)]
T_window_pts = [t for t in range(mx_T-length, mx_T)]

Q_x = [ q[1]['x'] for q in query.border.items() ]
Q_y = [ q[1]['y'] for q in query.border.items() ]

T_orig_x = [ t[1]['x'] for t in target.border.items() ]
T_orig_y = [ t[1]['y'] for t in target.border.items() ]

target.reposition_by_sample(T_window_pts, query, Q_window_pts)

T_shifted_x = [t[1]['x'] for t in target.border.items() ]
T_shifted_y = [t[1]['y'] for t in target.border.items() ]

plt.figure(figsize = [15,15])
plt.scatter( Q_x, Q_y, c = 'blue', s=16)
plt.scatter( T_orig_x, T_orig_y, c = 'orange', s=16)
plt.scatter( T_shifted_x, T_shifted_y, c = 'black', s=16)
#plt.savefig('placement_figs/temp_translation.png')
            
query.merge(target)

Q_merged_x = [ q[1]['x'] for q in query.border.items() ]
Q_merged_y = [ q[1]['y'] for q in query.border.items() ]

plt.figure(figsize = [15,15])
plt.scatter( Q_merged_x, Q_merged_y, c = 'blue', s=16)
#plt.savefig('placement_figs/temp_composite.png')